In [20]:
#!pip install PyPDF2
#nltk.download('punkt')
#nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\7oda\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [1]:
import PyPDF2
from PyPDF2 import PdfFileReader
import pandas as pd
import os
import nltk
from nltk.corpus import stopwords
import numpy as np
import ast
#from ast import literal_eval

In [2]:
def files():
    directory = "C:\\Users\\7oda\\New folder\\Query-Optimization\\files\\"
    files_name = []
    for sample in os.listdir(directory):
        files_name.append(sample)
    return files_name

In [3]:
def order(files_list):
    temp = 0
    temp_id = 0
    str_temp=''
    for i in range(len(files_list)):
        pdf = open("files/"+files_list[i],"rb")
        pdf_reader = PyPDF2.PdfFileReader(pdf)
        if(pdf_reader.numPages > temp):
            temp = pdf_reader.numPages 
            str_temp = files_list[i]
            del files_list[i]
            files_list.insert(0, str_temp)
            #print("found"+ files_list[i])
            
    return files_list

In [28]:
def reading():
    print("=============in the reading =============")

    #files_list =[]
    files_list = files()
    #make the bigger document in the begenning of the list
    files_list = order(files_list)
    
    #creating a dataframe to hold the docs
    df= pd.DataFrame()#

    for j in range(len(files_list)):
        page_list=[]
        print(files_list[j])
        pdf = open("files/"+files_list[j],"rb")
        pdf_reader = PyPDF2.PdfFileReader(pdf)

        print(pdf_reader.numPages)

        for i in range (pdf_reader.numPages):
            page = pdf_reader.getPage(i)
            pdf_words = page.extractText()
            page_list.append(pdf_words)
            #print(page_list)
        df[files_list[j]] = pd.Series(page_list)
        


    #closing the pdf file
    pdf.close()
    return df

In [29]:
def normalize(df):
    print("=============in the normalize =============")
    df['pages'] = df.index
    #this cell runs only one time
    # if you want to run again go back from the previous cell

    files_list = files()
    stop = stopwords.words('english')
    for i in  files_list:
        # convert to lower case
        df[i] = df[i].str.lower()
        # removing puncituations
        df[i] = df[i].str.replace('[^\w\s]','')
        #convert to string
        df[i] = df[i].apply(str)
    return df

In [35]:
def expanding(df):
    print("=============in the expanding =============")
    new_df=pd.DataFrame()
#    global word_list 
#    global index_list 
    #loop on each column or file
    for i in range (len(df.columns)-1):
        word_list = []
        index_list = []
        page_list = []
        index=0
        print("in the file #"+str(i))
        print("===================================")
        #loope on each cell or page
        for j in range(len(df.index)):
            ptr=-1
            #print(j)
            if (df.iloc[j][i] == 'nan'):
                break
            #loop on each element on the cell or character
            for k in range(len(df.iloc[j][i])):
                if (df.iloc[j][i][k] == '\n'):
                    index_list.append(index)
                    word_list.append(df.iloc[j][i][ptr+1:k])
                    page_list.append(df.iloc[j][len(df.columns)-1])
                    ptr = k
                    index=index+1
                    #print(k)
                    
        new_df[str(i)] = pd.Series(word_list)
        new_df['file '+ str(i)+' indexes'] = pd.Series(index_list)
        new_df['file '+ str(i)+' page numbers'] = pd.Series(page_list)
        print("length of word_list "+str(i)+" is  ---> "+str(len(word_list)))
        
    return new_df

In [31]:
def tokenize(new_df):
    print("=============in the tokenize=============")
    for i in  range(0,4,1):
        new_df.loc[:][str(i)] = new_df.loc[:][str(i)].apply(str)
        new_df.loc[:][str(i)] = new_df.loc[:][str(i)].apply(nltk.word_tokenize)
        #new_df.loc[:][str(i)] = new_df.loc[:][str(i)].apply(lambda x: [item for item in x if item not in stop])
    return new_df

In [77]:
def return_term_list(new_df):
    print("=============in the return_term_list =============")
    all_terms_list = [[]]
    
    #loop on each file
    for i in range (0, len(new_df.columns), 3):
        #loope on each cell or page
        counter = 0
        for j in range(len(new_df.index)):
            if (new_df.iloc[j][i] == ['nan'] or new_df.iloc[j][i] == []):
                continue
            for k in range(len(new_df.iloc[j][i])):
                term_list = []
                #term
                term_list.append(new_df.iloc[j][i][k])
                #doc_id
                term_list.append(i/3)
                #token_pos in the document
                term_list.append(new_df.iloc[j][i+1]+k+counter)
                if(k == 1 and len(new_df.iloc[j][i]) == 2):
                    counter +=1
                elif(k == len(new_df.iloc[j][i])-1):
                    counter += k 
                #page_id
                #term_list.append(new_df.iloc[j][i+2])
                
                #append each term to the all_term_list
                all_terms_list.append(term_list)
            
    return all_terms_list      
#        print(i)

In [100]:
def stop_words_removal(all_terms_list):
    new_terms_list = [[]]
    print("=============in the stop_words_removal=============")
    stop = stopwords.words('english')

    for i in range(len(all_terms_list)):
        if (all_terms_list[i] != []):
            if not all_terms_list[i][0] in stop:
                new_terms_list.append(all_terms_list[i])#.apply(lambda x: [item for item in x if item not in stop])
    return new_terms_list

In [103]:
#df = reading()
#df = normalize(df)
#df = expanding(df)
#df = tokenize(df)
#all_terms_list = return_term_list(df)
all_terms_list = stop_words_removal(all_terms_list)

=============in the stop_words_removal=============


In [ ]:
all_terms_list = stop_words_removal(all_terms_list)

In [72]:
df

,0,file 0 indexes,file 0 page numbers,1,file 1 indexes,file 1 page numbers,2,file 2 indexes,file 2 page numbers,3,file 3 indexes,file 3 page numbers
0,"[do, the]",0,0,[0],0.0,0.0,[helwanuniversity],0.0,0.0,"[a, true, story, about, why, we, sabotage, our...",0.0,0.0
1,[work],1,0,"[career, path, description, the, artificial, i...",1.0,0.0,[facultyofcomputerscience],1.0,0.0,"[feel, overwhelmed, set, aside, our, dreams, a...",1.0,0.0
2,"[do, the]",2,1,"[students, to, apply, ai, concepts, to, build,...",2.0,0.0,[andinformation],2.0,0.0,"[simply, be, ourselves, and, how, to, start, c...",2.0,0.0
3,[work],3,1,"[life, solutions]",3.0,0.0,[departmentofcomputerscience],3.0,0.0,"[new, york, times]",3.0,0.0
4,"[overcome, resistance, and, get, out, of, your...",4,1,"[this, career, path, introduces, students, to,...",4.0,0.0,[datastructuresspring2007],4.0,0.0,"[bestselling, author]",4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2479,[],2479,56,[nan],NaN,NaN,[nan],NaN,NaN,[nan],NaN,NaN
2480,"[numerous, civilizations, are, written, the, p...",2480,56,[nan],NaN,NaN,[nan],NaN,NaN,[nan],NaN,NaN
2481,"[martin, luther, king, jr]",2481,56,[nan],NaN,NaN,[nan],NaN,NaN,[nan],NaN,NaN
2482,"[beyond, vietnam, a, time, to, break, silence]",2482,56,[nan],NaN,NaN,[nan],NaN,NaN,[nan],NaN,NaN


In [75]:
print(df.iloc[9][1])

9


In [76]:
for v in df['0']:
    print(v)

['do', 'the']
['work']
['do', 'the']
['work']
['overcome', 'resistance', 'and', 'get', 'out', 'of', 'your', 'own', 'way', 'eld']
['author', 'of', 'the', 'war', 'of', 'art']
['also', 'by', 'steven', 'pres']
['eld']
['fiction']
['e', 'profession']
['killing', 'rommel']
['e', 'afghan', 'campaign']
['e', 'virtues', 'of', 'war']
['last', 'of', 'the', 'amazons']
['tides', 'of', 'war']
['gates', 'of', 'fire']
['e', 'legend', 'of', 'bagger', 'vance']
['n']
['ction']
['e', 'lions', 'gate']
['e', 'authentic', 'swing']
['turning', 'pro']
['e', 'warrior', 'ethos']
['e', 'war', 'of', 'art']
['do', 'the', 'work', '2011', 'steven', 'pres']
['eld']
['e', 'domino', 'project']
['published', 'by', 'do', 'you', 'zoom', 'inc', 'and', 'black', 'irish', 'entertainment', 'llc']
[]
['sign', 'up', 'for', 'updates', 'and', 'free', 'stu']
['at', 'wwwthedominoprojectcom']
['library', 'of', 'congress', 'cataloging', 'in', 'publication', 'data']
[]
['press']
['eld', 'steven', '1943']
['do']
['e', 'work', 'overcome',

['you', 'can', 'board', 'a', 'spaceship', 'to', 'pluto', 'and', 'settle', 'all', 'by', 'yourself', 'into']
[]
['a', 'perfect', 'artists', 'cottage', 'ten', 'zillion', 'miles', 'from', 'earth', 'resistance']
[]
['will', 'still', 'be', 'with', 'you']
['e', 'enemy', 'is', 'inside', 'you']
['principle', 'number', 'four']
['the', 'enemy', 'is', 'inside', 'you', 'but', 'it', 'is', 'not', 'you']
['e', 'fourth', 'axiom', 'of', 'resistance', 'is', 'that', 'the', 'enemy', 'is', 'inside', 'you']
['but', 'it', 'is', 'not', 'you']
['what', 'does', 'that', 'mean', 'it', 'means', 'you']
['are', 'not', 'to', 'blame', 'for', 'the', 'voices', 'of']
['resistance', 'you', 'hear', 'in', 'your', 'head']
['ey', 'are', 'not', 'your', 'fault', 'you', 'have', 'done', 'nothing', 'wrong', 'you']
['have', 'committed', 'no', 'sin', 'i', 'have', 'that', 'same', 'voice', 'in', 'my', 'head', 'so']
['steven', 'p']
['elddo']
['e', 'work']
['6263did', 'picasso', 'and', 'einstein', 'so', 'do', 'sarah', 'palin', 'and', 'la

['riage', 'and', 'blew', 'up', 'my', 'life', 'rather', 'than', 'plunge', 'a', 'sword', 'into', 'the']
[]
['heart', 'of', 'that', 'book', 'and', 'ship', 'it']
['it', 'took', 'me', 'seven', 'more', 'years', 'before', 'i', 'found', 'the', 'courage', 'to', 'face']
['that', 'dragon', 'againand', 'another', 'te']
['er', 'that', 'before', 'i', 'had']
['nally', 'learned', 'how', 'to', 'lay', 'him', 'out']
['heres', 'one', 'thing', 'i', 'can', 'tell', 'youand', 'you', 'can', 'take', 'this', 'to', 'the']
['bank']
['slay', 'that', 'dragon', 'once', 'and', 'he', 'will']
['never', 'have', 'power', 'over', 'you', 'again']
['steven', 'p']
['elddo']
['e', 'work']
['9697you', 'have', 'joined', 'an', 'elite', 'fraternity']
['whether', 'you', 'realize', 'it', 'or', 'not']
['by', 'dint', 'of', 'you']
['orts', 'and', 'your', 'perseverance', 'you', 'have', 'initi']
['ated', 'yourself', 'into', 'an', 'invisible', 'freemasonry', 'whose', 'members', 'are']
[]
['awarded', 'no', 'badges', 'or', 'insignia', 'shar

In [105]:
all_terms_list

[[],
 ['work', 0.0, 2],
 ['work', 0.0, 5],
 ['overcome', 0.0, 6],
 ['resistance', 0.0, 7],
 ['get', 0.0, 9],
 ['way', 0.0, 14],
 ['eld', 0.0, 15],
 ['author', 0.0, 16],
 ['war', 0.0, 19],
 ['art', 0.0, 21],
 ['also', 0.0, 22],
 ['steven', 0.0, 24],
 ['pres', 0.0, 25],
 ['eld', 0.0, 26],
 ['fiction', 0.0, 27],
 ['e', 0.0, 28],
 ['profession', 0.0, 29],
 ['killing', 0.0, 30],
 ['rommel', 0.0, 31],
 ['e', 0.0, 32],
 ['afghan', 0.0, 33],
 ['campaign', 0.0, 34],
 ['e', 0.0, 35],
 ['virtues', 0.0, 36],
 ['war', 0.0, 38],
 ['last', 0.0, 39],
 ['amazons', 0.0, 42],
 ['tides', 0.0, 43],
 ['war', 0.0, 45],
 ['gates', 0.0, 46],
 ['fire', 0.0, 48],
 ['e', 0.0, 49],
 ['legend', 0.0, 50],
 ['bagger', 0.0, 52],
 ['vance', 0.0, 53],
 ['n', 0.0, 54],
 ['ction', 0.0, 55],
 ['e', 0.0, 56],
 ['lions', 0.0, 57],
 ['gate', 0.0, 58],
 ['e', 0.0, 59],
 ['authentic', 0.0, 60],
 ['swing', 0.0, 61],
 ['turning', 0.0, 62],
 ['pro', 0.0, 63],
 ['e', 0.0, 64],
 ['warrior', 0.0, 65],
 ['ethos', 0.0, 66],
 ['e', 0.0,

In [102]:
len(all_terms_list)

19796

In [104]:
len(all_terms_list)

11289

In [13]:
#df.iloc[0][2]
#df

In [5]:
#df['pages'] = df.index
#this cell runs only one time
# if you want to run again go back from the previous cell

#files_list = files()
#stop = stopwords.words('english')
#for i in  files_list:
    # convert to lower case
  #  df[i] = df[i].str.lower()
    # removing puncituations
 #   df[i] = df[i].str.replace('[^\w\s]','')
    #convert to string
#    df[i] = df[i].apply(str)
    

    #splitting new lines on '\n'
#    for j in range(len(df.index)):
#        df[i][j] = df[i][j].replace('\r', ' ').split('\n')
    #df[i] = df[i].apply(literal_eval)    
    #df[i] = df[i].apply(str)

    #df.explode(i)

# convert list of pd.Series then stack it
#    df = (df
     #.set_index(['issue_key','date','pkey','case_count'])['component']
#     .apply(pd.Series)
#     .stack()
#     .reset_index()
#     .reset_index(level=0, drop=True))
     #.rename(columns={0:'component'}))

    #convert to string
    #df[i] = df[i].apply(str)
    
    # tokenize
    #df[i] = df[i].apply(nltk.word_tokenize)
    
    # removing stop words
    #df[i].apply(lambda x: [item for item in x if item not in stop])
    
    #df[i]= df[i].str.splitlines()
    #df[i].apply(lambda x: (word for word in x.str.splitlines()))
    

In [165]:
#df.iloc[1][1]

In [166]:
#df.iloc[1][1][8:12]
#list0

In [107]:
#len(df.columns)-1
for i in range(5, 3, 2):
    print (i)
    if (i == 2):
        break
#df.iloc[50][2] == 'nan'

0
1
2


In [167]:
#df

In [97]:
new_df = expanding(df)

0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
length of word_list 0 is  ---> 2484
1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
length of word_list 1 is  ---> 300
2
0
1
length of word_list 2 is  ---> 56
3
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
length of word_list 3 is  ---> 554


In [168]:
#pd.set_option("display.max_rows", None, "display.max_columns", None)

#print(df.iloc[0][2])

In [169]:
#pd.set_option("display.max_rows", None, "display.max_columns", None)
#print(new_df)


In [16]:
#new_df=new_df.reset_index(drop=True)

In [98]:
new_df

,0,file 0 indexes,file 0 page numbers,1,file 1 indexes,file 1 page numbers,2,file 2 indexes,file 2 page numbers,3,file 3 indexes,file 3 page numbers
0,do the,0,0,0,0.0,0.0,helwanuniversity,0.0,0.0,a true story about why we sabotage ourselves,0.0,0.0
1,work,1,0,career path description the artificial intelli...,1.0,0.0,facultyofcomputerscience,1.0,0.0,feel overwhelmed set aside our dreams and lack...,1.0,0.0
2,do the,2,1,students to apply ai concepts to build real,2.0,0.0,andinformation,2.0,0.0,simply be ourselves and how to start choosin...,2.0,0.0
3,work,3,1,life solutions,3.0,0.0,departmentofcomputerscience,3.0,0.0,new york times,3.0,0.0
4,overcome resistance and get out of your own wa...,4,1,this career path introduces students to basic ...,4.0,0.0,datastructuresspring2007,4.0,0.0,bestselling author,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2479,,2479,56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2480,numerous civilizations are written the patheti...,2480,56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2481,martin luther king jr,2481,56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2482,beyond vietnam a time to break silence,2482,56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [170]:
#df.loc[10][1]

In [124]:
for i in  range(1):
    
    print(i)

0


In [105]:
stop = stopwords.words('english')
for i in  range(0,4,1):
    new_df.loc[:][str(i)] = new_df.loc[:][str(i)].apply(str)
    new_df.loc[:][str(i)] = new_df.loc[:][str(i)].apply(nltk.word_tokenize)
    new_df.loc[:][str(i)] = new_df.loc[:][str(i)].apply(lambda x: [item for item in x if item not in stop])


In [67]:
#stop

In [148]:
new_df.iloc[553][9]

['phone', '8599807900']

In [146]:
new_df.iloc[400][3] == ['nan']

True

In [79]:
len(new_df.iloc[3][3])

2

In [80]:
lista = []
lista.append('a')
lista.append(3)

In [81]:
lista

['a', 3]

In [137]:
simple_list = return_term_list(new_df)

In [142]:
simple_list[11288]

['8599807900', 9, 553.0, 31.0]

In [69]:
#b = pd.DataFrame(df["Artificial Intelligince.pdf"].str.split('\n').values.tolist()).stack()
#b = DataFrame(a.var1.str.split(',').values.tolist(), index=a.var2).stack()
#b = b.reset_index()[[0, 'var2']]

In [29]:
#df.iloc[0][0]

In [171]:
#df

In [267]:
m=0

In [268]:
def mm():
    return m+1

In [269]:
mm

<function __main__.mm()>

In [270]:
m

0

In [271]:
print(mm)

<function mm at 0x0000026936404DC8>
